# Modèles basés sur des SVM
   Avec benchmark des méthodes de recalibrage.

**(1) Importation des librairies**

In [1]:
import os
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
import sys 
import fastparquet
import dask_ml 
from dask import delayed
from sklearn.metrics import *
import scikitplot as skplt
from dask_ml.preprocessing import StandardScaler
import pickle
import dask
from sklearn.tree import DecisionTreeClassifier

**(2) Import de l'algorithme SVM**

In [2]:
from sklearn import svm
from sklearn.svm import SVC

In [3]:
#pip install dask.ml

In [4]:
#pip install scikit-plot

# I. Préparation

**(1) variables explicatives, variable à prédire et centrage-réduction**

In [5]:
import os
os.chdir("C:/Fouille-donnees-massives")

In [6]:
# Dataframe d’apprentissage brut
df_train = dd.read_parquet("train_fast")
df_train.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 12 entries, FlagImpaye to CA3TR
dtypes: float64(7), int64(5)

In [7]:
# Dataframe de test
df_test = dd.read_parquet("test_fast")
df_test.head()

,FlagImpaye,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR
3899362,0,60.00,0,302,0.000000,0.000000,0.000000,52.076034,0,1,4.000000,0.00
3899363,0,25.00,0,547,0.000000,0.000000,0.000000,52.076034,0,1,4.000000,0.00
3899364,0,32.00,0,152,0.000000,0.000000,0.000000,52.076034,0,1,4.000000,0.00
3899365,0,20.88,0,393,0.007590,0.182221,0.002092,34.118277,4,1,1.539155,71.51
3899366,0,20.21,0,152,0.018994,-1.024399,0.005682,52.076034,0,1,4.000000,0.00


In [8]:
# Définir les variables explicatives et la variable cible 
X_train = df_train.drop(["FlagImpaye"], axis = "columns")
X_test = df_test.drop(["FlagImpaye"], axis = "columns")

y_train = df_train["FlagImpaye"]
y_test = df_test["FlagImpaye"]

In [9]:
X_train.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 11 entries, Montant to CA3TR
dtypes: float64(7), int64(4)

In [10]:
# Instanciation de StandardScaler pour centrer-réduire les données
scaler = StandardScaler()

In [11]:
# Centrer-réduire le dataframe d'apprentissage
X_train = scaler.fit_transform(X_train)
X_train.head()

,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR
0,-0.420302,-0.581824,1.017656,-0.084590,-0.239505,-0.275801,0.818043,-0.043817,-0.194165,-0.451325,-0.202861
1,-0.420302,-0.581824,1.017656,-0.084590,-0.239505,-0.275801,0.818043,-0.043812,4.975724,-0.451325,0.038808
2,-0.022127,-0.581824,1.008573,-0.084590,-0.239505,-0.275801,0.818043,-0.043817,-0.194165,-0.451325,-0.202861
3,-0.057565,1.545332,-0.272152,-0.084590,-0.239505,-0.275801,0.906727,-0.043817,-0.194165,-0.451325,-0.202861
4,-0.347310,-0.581824,1.008573,-0.084434,0.637044,-0.273204,0.818043,-0.043812,-0.194165,-0.451325,0.702346


In [40]:
# Centrer-réduire les dataframe de test
X_test = scaler.fit_transform(X_test)
X_test.head()

,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR
3899362,-0.016999,0.0,0.265719,-0.080269,-0.043960,-0.317899,0.774799,-0.036921,-0.18432,-0.568544,-0.132257
3899363,-0.277531,0.0,1.427785,-0.080269,-0.043960,-0.317899,0.774799,-0.036921,-0.18432,-0.568544,-0.132257
3899364,-0.225425,0.0,-0.445750,-0.080269,-0.043960,-0.317899,0.774799,-0.036921,-0.18432,-0.568544,-0.132257
3899365,-0.308200,0.0,0.697344,-0.080074,-0.023966,-0.314458,-0.277090,-0.036897,-0.18432,-5.906429,1.259186
3899366,-0.313187,0.0,-0.445750,-0.079781,-0.156360,-0.308551,0.774799,-0.036921,-0.18432,-0.568544,-0.132257


**(2) Recodage des modalités de Y en -1 et 1 (scores de séparation de la marge)**

In [13]:
# On recode en -1 et 1 pour les besoins de ces algorithmes qui affectent un score de -1 ou 1
y_train = y_train.replace({1: -1, 0 : 1})
y_Test = y_test.replace({1: -1, 0 : 1})

**(2) Maximisation de la marge**

In [14]:
def perte(Montant):
    if Montant<=20:
        Montant=Montant*0
    elif 20<Montant<=50:
        Montant=0.2*Montant
    elif 50<Montant<=100:
        Montant=0.3*Montant
    elif 100<Montant<=200:
        Montant=0.5*Montant
    elif Montant>200:
        Montant=0.8*Montant
    return(Montant)

In [15]:
def Calcul_Marge(Montant, yReel, yPred):
    # Création de dfmerge
    dfmerge = pd.concat([Montant, yReel], axis=1)
    dfmerge["Ypred"] = yPred
    
    # Création de la variable Marge
    # Création d'une colonne Marge égale au Montant
    dfmerge["Marge"] = dfmerge["Montant"]
    #Calcul du gain ou de la perte selon la différence entre réel et prédiction
    dfmerge.loc[((dfmerge["FlagImpaye"] == 1) & (dfmerge["Ypred"] == 1)), "Marge"] = 0
    dfmerge.loc[((dfmerge["FlagImpaye"] == 0) & (dfmerge["Ypred"] == 1)), "Marge"] = 0.7 * 0.05 * dfmerge["Montant"]
    dfmerge.loc[((dfmerge["FlagImpaye"] == 1) & (dfmerge["Ypred"] == 0)), "Marge"] = dfmerge["Montant"].apply(lambda x: -perte(x))
    dfmerge.loc[((dfmerge["FlagImpaye"] == 0) & (dfmerge["Ypred"] == 0)), "Marge"] = 0.05 * dfmerge["Montant"]
    
    # Calcul du Marge_totale
    Marge_totale = dfmerge["Marge"].sum()
    
    return Marge_totale

**(3) Modèles et performances**

In [16]:
# Fonction de déploiement d'un modèle sur les données de test

# Création de la fonction de performance d'un modèle sur les données de test

def performances(modele, X_test, y_test, scale):
    
    X_test_ = X_test
    
    print("-------------------------------------------------------")
   
    
    if scale == True:
        X_test = scaler.transform(X_test) # scaler a été entrainé sur les données d'apprentissage
        
    # Affichage du modèle 
    print('Modèle :\n', modele)
    print("-------------------------------------------------------")
    # Prédiction
    y_pred = delayed(modele.predict)(X_test).compute()
    # Prédiction des scores
    y_score = delayed(modele.predict_proba)(X_test).compute()
    
    # Estimateurs, matrice de confusion et AUC
    cm = delayed(confusion_matrix)(y_test, y_pred).compute()
    cr = delayed(classification_report)(y_test, y_pred).compute()
    auc = delayed(roc_auc_score)(y_test, y_pred).compute()
    f1 = delayed(f1_score)(y_test, y_pred).compute()
   
    print('F1_score :\n', f1)
    print("-------------------------------------------------------")
    print('Estimateurs :\n', cr)
    print("-------------------------------------------------------")
    print('Matrice de confusion :\n', cm)
    print("-------------------------------------------------------")
    print('Auc Score :\n', auc)
    print("-------------------------------------------------------")
    
    #Calcul de la Marge
    Marge_totale = Calcul_Marge(Montant = X_test_["Montant"].compute(), yReel = y_test.compute(), yPred = y_pred)
    print("Marge = " + str(round(Marge_totale, 2)) + " euros")
    
    print("-------------------------------------------------------")
    
    # Courbe ROC
    skplt.metrics.plot_roc(y_test, y_score, classes_to_plot = 1, plot_micro = False, plot_macro = False)
    # Courbe Précision-Rappel
    skplt.metrics.plot_precision_recall(y_test, y_score, classes_to_plot = 1, plot_micro = False)

    #return y_score,Marge_totale,auc,cr,cm

Si la courbe ROC tient la route avec un critère AUC élevé, cela signifie que l'apprentissage a bien fonctionné Si le F1_score est très mauvais, cela signifie alors que c'est le processus d'affectation qui n'a pas fonctionné.

# III. SVM

# A. avec paramétrage de class_weight

Nous faisons l'économie de tester le noyau linéaire, qui n'est pas adapté à la complexité du modèle.
Nous testons le noyau polynomial et le noyau gaussien RBF qui est mieux adapté pour résoudre les problèmes de séparabilité linéaire. 

Plusieurs tentatives / h -> pas de sortie : approche KO

In [17]:
# Instanciation 
#SVM avec un noyau polynomial de degré 5 #
#SVM= SVC(kernel="poly", class_weight='balanced', degree=5,random_state=0)

# Fit your pipeline onto your training set (centré-réduit) and obtain predictions by fitting the model onto the test data
#classifier1 = delayed(SVM.fit)(X_train, y_train).compute()

In [18]:
#classifier1

In [19]:
#performances(classifier1, X_test, y_test,scale=True)

In [20]:
# Instanciation 
#SVM avec un noyau gaussien pour résoudre les problèmes de non séparabilité linéaire
#SVM= SVC(kernel='rbf', class_weight='balanced',gamma=0.1)

# Fit your pipeline onto your training set (centré-réduit) and obtain predictions by fitting the model onto the test data
#classifier2 = delayed(SVM.fit)(X_train, y_train).compute()


In [21]:
#classifier2

In [22]:
#performances(classifier2, X_test, y_test,scale=True)

# B. avec borderline SMOTE

In [23]:
# Dataframe d’apprentissage recalibré avec borderline SMOTE (k=3 ; k'=2) et centré-réduit
df_train = dd.read_parquet("train_borderS")
X_train = df_train.drop(["FlagImpaye"], axis = "columns")
y_train = df_train["FlagImpaye"]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [24]:
y_train = y_train.replace({1: -1, 0 : 1})
y_Test = y_test.replace({1: -1, 0 : 1})

In [25]:
# Instanciation #Modèle pas adapté
#SVM avec un noyau polynomial de degré 5
#Mobilisation de tous les processeurs (n_jobs=-1)
#SVM= SVC(kernel="poly", degree=5,random_state=0,gamma=0.1,n_jobs=-1)

# Fit your pipeline onto your training set (centré-réduit) and obtain predictions by fitting the model onto the test data
#classifier1b = delayed(SVM.fit)(X_train, y_train).compute()

In [26]:
#classifier1b

In [27]:
#performances(classifier1b, X_test, y_test,scale=True)

In [28]:
# Instanciation 
#SVM avec un noyau gaussien pour résoudre les problèmes de non séparabilité linéaire

#SVM= SVC(kernel='rbf',gamma=0.1)

# Fit your pipeline onto your training set (centré-réduit) and obtain predictions by fitting the model onto the test data
#classifier2b = delayed(SVM.fit)(X_train, y_train).compute()

In [29]:
#classifier2b

In [30]:
#performances(classifier2b, X_test, y_test,scale=True)

In [31]:
#importances=pd.Series(SVM.feature_importances_, index= X_Test.columns)
#sorted_importances=importances.sort_values()
#sorted_importances.plot(kind='barh',color='blue')

**(2) Tentative avec appproximation du noyau**

principe : approximer le modèle en faisant travailler le modèle sur un sous-échantillon

objectif : réussir à faire apprendre le modèle en moins d'1 heure

In [32]:
from sklearn.kernel_approximation import Nystroem
from sklearn.pipeline import make_pipeline

In [43]:
erreur 
from sklearn.pipeline import Pipeline
#Instantiation avec approximation du kernel
svm_rbf=SVC(random_state=0)
transform=Nystroem(kernel='rbf',gamma=0.1,random_state=0)

# Pipeline
clf1 = Pipeline([('Nystroem', transform), ('svm_rdf', svm_rbf)])

# Fit your pipeline onto your training set and obtain predictions by fitting the model onto the test data 
classifier1=delayed(clf1.fit)(X_train, y_train).compute() 

MemoryError: Unable to allocate 5.78 GiB for an array with shape (7751880, 100) and data type float64

In [ ]:
classifier1

In [ ]:
performances(classifier1, X_test, y_test,scale=False)

Méthode 2

In [ ]:
#Instantiation avec approximation du kernel
svm_rbf=SVC(random_state=0)
transform=Nystroem(kernel='rbf',gamma=0.1,random_state=0)

# Pipeline
pipeline_svm = make_pipeline(transform, svm_rbf)
# Entraînement
fit_svm_rbf = delayed(pipeline_svm.fit)(X_train)

# Prédiction
pred_svm_rbf = delayed(fit_svm_rbf.predict)(X_test)
y_pred_svm_rbf = pred_svm_rbf.compute()

In [ ]:
performances(fit_svm_rbf, X_test, y_test,scale=False)